In [1]:
from keras.layers.convolutional import Conv2D
from keras.layers import Dense, Flatten
from keras.optimizers import RMSprop
from keras.models import Sequential
from skimage.transform import resize
from skimage.color import rgb2gray
from collections import deque
from keras import backend as K
import tensorflow as tf
import numpy as np
import random
import gym

Using TensorFlow backend.
C:\Users\student\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\student\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\student\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\student\Anaconda3\lib\site-packages\tensorflow\python\framework\d

In [2]:
tf.__version__

'1.13.1'

In [3]:
EPISODES = 50000

In [4]:
class DQNAgent:
    def __init__(self, action_size):
        self.render = False
        self.load_model = False
        # 상태와 행동의 크기 정의
        self.state_size = (84, 84, 4)
        self.action_size = action_size
        # DQN 하이퍼파라미터
        self.epsilon = 1.
        self.epsilon_start, self.epsilon_end = 1.0, 0.1
        self.exploration_steps = 1000000.
        self.epsilon_decay_step = (self.epsilon_start - self.epsilon_end) \
                                  / self.exploration_steps
        self.batch_size = 32
        self.train_start = 50000
        self.update_target_rate = 10000
        self.discount_factor = 0.99
        # 리플레이 메모리, 최대 크기 400000
        self.memory = deque(maxlen=400000)
        self.no_op_steps = 30
        # 모델과 타겟모델을 생성하고 타겟모델 초기화
        self.model = self.build_model()
        self.target_model = self.build_model()
        self.update_target_model()

        self.optimizer = self.optimizer()

        # 텐서보드 설정
        self.sess = tf.InteractiveSession()
        K.set_session(self.sess)

        self.avg_q_max, self.avg_loss = 0, 0
        self.summary_placeholders, self.update_ops, self.summary_op = \
            self.setup_summary()
        self.summary_writer = tf.summary.FileWriter(
            'summary/breakout_dqn', self.sess.graph)
        self.sess.run(tf.global_variables_initializer())

        if self.load_model:
            self.model.load_weights("./save_model/breakout_dqn.h5")

    # Huber Loss를 이용하기 위해 최적화 함수를 직접 정의
    def optimizer(self):
        a = K.placeholder(shape=(None,), dtype='int32')
        y = K.placeholder(shape=(None,), dtype='float32')

        prediction = self.model.output

        a_one_hot = K.one_hot(a, self.action_size)
        q_value = K.sum(prediction * a_one_hot, axis=1)
        error = K.abs(y - q_value)

        quadratic_part = K.clip(error, 0.0, 1.0)
        linear_part = error - quadratic_part
        loss = K.mean(0.5 * K.square(quadratic_part) + linear_part)

        optimizer = RMSprop(lr=0.00025, epsilon=0.01)
        updates = optimizer.get_updates(self.model.trainable_weights, [], loss)
        train = K.function([self.model.input, a, y], [loss], updates=updates)

        return train

    # 상태가 입력, 큐함수가 출력인 인공신경망 생성
    def build_model(self):
        model = Sequential()
        model.add(Conv2D(32, (8, 8), strides=(4, 4), activation='relu',
                         input_shape=self.state_size))
        model.add(Conv2D(64, (4, 4), strides=(2, 2), activation='relu'))
        model.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu'))
        model.add(Flatten())
        model.add(Dense(512, activation='relu'))
        model.add(Dense(self.action_size))
        model.summary()
        return model

    # 타겟 모델을 모델의 가중치로 업데이트
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    # 입실론 탐욕 정책으로 행동 선택
    def get_action(self, history):
        history = np.float32(history / 255.0)
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        else:
            q_value = self.model.predict(history)
            return np.argmax(q_value[0])

    # 샘플 <s, a, r, s'>을 리플레이 메모리에 저장
    def append_sample(self, history, action, reward, next_history, dead):
        self.memory.append((history, action, reward, next_history, dead))

    # 리플레이 메모리에서 무작위로 추출한 배치로 모델 학습
    def train_model(self):
        if self.epsilon > self.epsilon_end:
            self.epsilon -= self.epsilon_decay_step

        mini_batch = random.sample(self.memory, self.batch_size)

        history = np.zeros((self.batch_size, self.state_size[0],
                            self.state_size[1], self.state_size[2]))
        next_history = np.zeros((self.batch_size, self.state_size[0],
                                 self.state_size[1], self.state_size[2]))
        target = np.zeros((self.batch_size,))
        action, reward, dead = [], [], []

        for i in range(self.batch_size):
            history[i] = np.float32(mini_batch[i][0] / 255.)
            next_history[i] = np.float32(mini_batch[i][3] / 255.)
            action.append(mini_batch[i][1])
            reward.append(mini_batch[i][2])
            dead.append(mini_batch[i][4])

        target_value = self.target_model.predict(next_history)

        for i in range(self.batch_size):
            if dead[i]:
                target[i] = reward[i]
            else:
                target[i] = reward[i] + self.discount_factor * \
                                        np.amax(target_value[i])

        loss = self.optimizer([history, action, target])
        self.avg_loss += loss[0]

    # 각 에피소드 당 학습 정보를 기록
    def setup_summary(self):
        episode_total_reward = tf.Variable(0.)
        episode_avg_max_q = tf.Variable(0.)
        episode_duration = tf.Variable(0.)
        episode_avg_loss = tf.Variable(0.)

        tf.summary.scalar('Total Reward/Episode', episode_total_reward)
        tf.summary.scalar('Average Max Q/Episode', episode_avg_max_q)
        tf.summary.scalar('Duration/Episode', episode_duration)
        tf.summary.scalar('Average Loss/Episode', episode_avg_loss)

        summary_vars = [episode_total_reward, episode_avg_max_q,
                        episode_duration, episode_avg_loss]
        summary_placeholders = [tf.placeholder(tf.float32) for _ in
                                range(len(summary_vars))]
        update_ops = [summary_vars[i].assign(summary_placeholders[i]) for i in
                      range(len(summary_vars))]
        summary_op = tf.summary.merge_all()
        return summary_placeholders, update_ops, summary_op


In [5]:
def pre_processing(observe):
    processed_observe = np.uint8(
        resize(rgb2gray(observe), (84, 84), mode='constant') * 255)
    return processed_observe

In [ ]:
if __name__ == "__main__":
    # 환경과 DQN 에이전트 생성
    env = gym.make('BreakoutDeterministic-v4')
    agent = DQNAgent(action_size=3)

    scores, episodes, global_step = [], [], 0

    for e in range(EPISODES):
        done = False
        dead = False

        step, score, start_life = 0, 0, 5
        observe = env.reset()

        for _ in range(random.randint(1, agent.no_op_steps)):
            observe, _, _, _ = env.step(1)

        state = pre_processing(observe)
        history = np.stack((state, state, state, state), axis=2)
        history = np.reshape([history], (1, 84, 84, 4))

        while not done:
            if agent.render:
                env.render()
            global_step += 1
            step += 1

            # 바로 전 4개의 상태로 행동을 선택
            action = agent.get_action(history)
            # 1: 정지, 2: 왼쪽, 3: 오른쪽
            if action == 0:
                real_action = 1
            elif action == 1:
                real_action = 2
            else:
                real_action = 3

            # 선택한 행동으로 환경에서 한 타임스텝 진행
            observe, reward, done, info = env.step(real_action)
            # 각 타임스텝마다 상태 전처리
            next_state = pre_processing(observe)
            next_state = np.reshape([next_state], (1, 84, 84, 1))
            next_history = np.append(next_state, history[:, :, :, :3], axis=3)

            agent.avg_q_max += np.amax(
                agent.model.predict(np.float32(history / 255.))[0])
            
            if start_life > info['ale.lives']:
                dead = True
                start_life = info['ale.lives']

            reward = np.clip(reward, -1., 1.)
            # 샘플 <s, a, r, s'>을 리플레이 메모리에 저장 후 학습
            agent.append_sample(history, action, reward, next_history, dead)

            if len(agent.memory) >= agent.train_start:
                agent.train_model()

            # 일정 시간마다 타겟모델을 모델의 가중치로 업데이트
            if global_step % agent.update_target_rate == 0:
                agent.update_target_model()

            score += reward

            if dead:
                dead = False
            else:
                history = next_history

            if done:
                # 각 에피소드 당 학습 정보를 기록
                if global_step > agent.train_start:
                    stats = [score, agent.avg_q_max / float(step), step,
                             agent.avg_loss / float(step)]
                    for i in range(len(stats)):
                        agent.sess.run(agent.update_ops[i], feed_dict={
                            agent.summary_placeholders[i]: float(stats[i])
                        })
                    summary_str = agent.sess.run(agent.summary_op)
                    agent.summary_writer.add_summary(summary_str, e + 1)

                print("episode:", e, "  score:", score, "  memory length:",
                      len(agent.memory), "  epsilon:", agent.epsilon,
                      "  global_step:", global_step, "  average_q:",
                      agent.avg_q_max / float(step), "  average loss:",
                      agent.avg_loss / float(step))

                agent.avg_q_max, agent.avg_loss = 0, 0

        # 1000 에피소드마다 모델 저장
        if e % 1000 == 0:
            agent.model.save_weights("./save_model/breakout_dqn.h5")

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 20, 20, 32)        8224      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 9, 9, 64)          32832     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
flatten_3 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               1606144   
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 1539      
Total params: 1,685,667
Trainable params: 1,685,667
Non-trainable params: 0
____________________________________________

C:\Users\student\Anaconda3\lib\site-packages\tensorflow\python\client\session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


episode: 0   score: 2.0   memory length: 187   epsilon: 1.0   global_step: 187   average_q: 0.055928201758287804   average loss: 0.0
episode: 1   score: 0.0   memory length: 324   epsilon: 1.0   global_step: 324   average_q: 0.05418188086826436   average loss: 0.0
episode: 2   score: 1.0   memory length: 456   epsilon: 1.0   global_step: 456   average_q: 0.05448061377374512   average loss: 0.0
episode: 3   score: 0.0   memory length: 560   epsilon: 1.0   global_step: 560   average_q: 0.054643740268567435   average loss: 0.0
episode: 4   score: 3.0   memory length: 806   epsilon: 1.0   global_step: 806   average_q: 0.05326916284980328   average loss: 0.0
episode: 5   score: 2.0   memory length: 992   epsilon: 1.0   global_step: 992   average_q: 0.053562487004905616   average loss: 0.0
episode: 6   score: 3.0   memory length: 1251   epsilon: 1.0   global_step: 1251   average_q: 0.0540279652419928   average loss: 0.0
episode: 7   score: 0.0   memory length: 1356   epsilon: 1.0   global_st

episode: 61   score: 0.0   memory length: 10190   epsilon: 1.0   global_step: 10190   average_q: 0.05450849422942037   average loss: 0.0
episode: 62   score: 1.0   memory length: 10346   epsilon: 1.0   global_step: 10346   average_q: 0.05396023337753155   average loss: 0.0
episode: 63   score: 0.0   memory length: 10468   epsilon: 1.0   global_step: 10468   average_q: 0.054362713985267236   average loss: 0.0
episode: 64   score: 0.0   memory length: 10582   epsilon: 1.0   global_step: 10582   average_q: 0.05441282408540709   average loss: 0.0
episode: 65   score: 0.0   memory length: 10677   epsilon: 1.0   global_step: 10677   average_q: 0.05412467890664151   average loss: 0.0
episode: 66   score: 0.0   memory length: 10789   epsilon: 1.0   global_step: 10789   average_q: 0.054213654748829346   average loss: 0.0
episode: 67   score: 0.0   memory length: 10916   epsilon: 1.0   global_step: 10916   average_q: 0.054336711647003655   average loss: 0.0
episode: 68   score: 3.0   memory leng

episode: 121   score: 0.0   memory length: 19256   epsilon: 1.0   global_step: 19256   average_q: 0.05443866563037159   average loss: 0.0
episode: 122   score: 1.0   memory length: 19394   epsilon: 1.0   global_step: 19394   average_q: 0.05500205509040667   average loss: 0.0
episode: 123   score: 3.0   memory length: 19636   epsilon: 1.0   global_step: 19636   average_q: 0.05353745882791921   average loss: 0.0
episode: 124   score: 2.0   memory length: 19839   epsilon: 1.0   global_step: 19839   average_q: 0.05296144643660836   average loss: 0.0
episode: 125   score: 0.0   memory length: 19967   epsilon: 1.0   global_step: 19967   average_q: 0.05426781336427666   average loss: 0.0
episode: 126   score: 0.0   memory length: 20072   epsilon: 1.0   global_step: 20072   average_q: 0.05437924567432631   average loss: 0.0
episode: 127   score: 0.0   memory length: 20215   epsilon: 1.0   global_step: 20215   average_q: 0.05431485988877036   average loss: 0.0
episode: 128   score: 1.0   memory

...

episode: 1044   score: 2.0   memory length: 166746   epsilon: 0.8949277000034593   global_step: 166746   average_q: 1.0451815567206388   average loss: 0.3329703099366836
episode: 1045   score: 1.0   memory length: 166907   epsilon: 0.8947828000034641   global_step: 166907   average_q: 1.106299769063914   average loss: 0.32104303532227424
episode: 1046   score: 3.0   memory length: 167121   epsilon: 0.8945902000034704   global_step: 167121   average_q: 1.1540144670789487   average loss: 0.3327010553520582
episode: 1047   score: 0.0   memory length: 167246   epsilon: 0.8944777000034742   global_step: 167246   average_q: 1.1409078512191773   average loss: 0.3255572577842486
episode: 1048   score: 1.0   memory length: 167398   epsilon: 0.8943409000034787   global_step: 167398   average_q: 1.0267618267159713   average loss: 0.3571033328386387
episode: 1049   score: 2.0   memory length: 167583   epsilon: 0.8941744000034841   global_step: 167583   average_q: 1.0318943258878348   average loss:

episode: 1093   score: 1.0   memory length: 174373   epsilon: 0.8880634000036853   global_step: 174373   average_q: 1.0992108552079451   average loss: 0.3537842723968173
episode: 1094   score: 0.0   memory length: 174496   epsilon: 0.887952700003689   global_step: 174496   average_q: 1.0330001929911172   average loss: 0.34055840287551403
episode: 1095   score: 0.0   memory length: 174621   epsilon: 0.8878402000036927   global_step: 174621   average_q: 1.034287308692932   average loss: 0.33064608132155954
episode: 1096   score: 0.0   memory length: 174728   epsilon: 0.8877439000036959   global_step: 174728   average_q: 1.0356276024167783   average loss: 0.3322440319468017
episode: 1097   score: 4.0   memory length: 174966   epsilon: 0.8875297000037029   global_step: 174966   average_q: 0.9849931835627356   average loss: 0.32270109755406146
episode: 1098   score: 1.0   memory length: 175149   epsilon: 0.8873650000037083   global_step: 175149   average_q: 0.9871932261628531   average loss

episode: 1142   score: 3.0   memory length: 182910   epsilon: 0.8803801000039383   global_step: 182910   average_q: 1.0625706432167619   average loss: 0.3337345204964831
episode: 1143   score: 2.0   memory length: 183092   epsilon: 0.8802163000039437   global_step: 183092   average_q: 1.0615934906425057   average loss: 0.32924670694427066
episode: 1144   score: 0.0   memory length: 183212   epsilon: 0.8801083000039472   global_step: 183212   average_q: 1.067831673224767   average loss: 0.3234586868038605
episode: 1145   score: 0.0   memory length: 183321   epsilon: 0.8800102000039505   global_step: 183321   average_q: 1.0663022787199108   average loss: 0.33441121357109277
episode: 1146   score: 0.0   memory length: 183427   epsilon: 0.8799148000039536   global_step: 183427   average_q: 1.0312047443299923   average loss: 0.32172698668676636
episode: 1147   score: 1.0   memory length: 183557   epsilon: 0.8797978000039575   global_step: 183557   average_q: 1.004210870541059   average loss

episode: 1191   score: 0.0   memory length: 190071   epsilon: 0.8739352000041505   global_step: 190071   average_q: 1.1848138322924624   average loss: 0.3778725874110005
episode: 1192   score: 1.0   memory length: 190244   epsilon: 0.8737795000041556   global_step: 190244   average_q: 1.1838510036468506   average loss: 0.33948192164126534
episode: 1193   score: 0.0   memory length: 190345   epsilon: 0.8736886000041586   global_step: 190345   average_q: 1.1166451638287838   average loss: 0.327459192067199
episode: 1194   score: 2.0   memory length: 190525   epsilon: 0.8735266000041639   global_step: 190525   average_q: 1.0896991087330712   average loss: 0.3462124035258999
episode: 1195   score: 2.0   memory length: 190702   epsilon: 0.8733673000041692   global_step: 190702   average_q: 1.1644537388268164   average loss: 0.3190908431582611
episode: 1196   score: 0.0   memory length: 190801   epsilon: 0.8732782000041721   global_step: 190801   average_q: 1.1732455118738039   average loss:

episode: 1240   score: 0.0   memory length: 197997   epsilon: 0.8668018000043853   global_step: 197997   average_q: 0.9982359532047721   average loss: 0.3598167362525003
episode: 1241   score: 0.0   memory length: 198104   epsilon: 0.8667055000043885   global_step: 198104   average_q: 1.000250659256338   average loss: 0.31239511592575053
episode: 1242   score: 2.0   memory length: 198272   epsilon: 0.8665543000043935   global_step: 198272   average_q: 0.9994468014864695   average loss: 0.3396010515452816
episode: 1243   score: 1.0   memory length: 198446   epsilon: 0.8663977000043986   global_step: 198446   average_q: 0.9447036090938524   average loss: 0.3277892053568697
episode: 1244   score: 6.0   memory length: 198780   epsilon: 0.8660971000044085   global_step: 198780   average_q: 0.9924375664688156   average loss: 0.3352926848147302
episode: 1245   score: 0.0   memory length: 198880   epsilon: 0.8660071000044115   global_step: 198880   average_q: 1.0172085654735565   average loss: